<a href="https://colab.research.google.com/github/sonalsingh1806/MachineLearningProjects/blob/main/ProjectAlpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install huggingface_hub
from huggingface_hub import login
login()  # enter your token


In [2]:
from google.colab import files
uploaded = files.upload()

Saving Design Patterns.pptx.pdf to Design Patterns.pptx.pdf


In [14]:
import streamlit as st
from utils import storage
from sidebar import render_sidebar
import json

st.set_page_config(page_title="Let's talk", page_icon="💬", layout="wide")

# --- 💬 Chatbot Section ---
st.title("💬 Let's Talk")
st.write("Talk to your AI teaching assistant on any topic, ask for explanations of rules, useful vocabulary, or exercises.")
st.write("Save any new words to your vocabulary list in the side panel.")
st.write("Press 'Quiz!' to get exercises for practicing random words from your vocabulary list.")

ModuleNotFoundError: No module named 'streamlit'

In [3]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00


In [4]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_file):
    reader = PdfReader(pdf_file)
    return "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

text = extract_text_from_pdf("Design Patterns.pptx.pdf")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.create_documents([text])
print(f"Split blog post into {len(chunks)} sub-documents.")


Split blog post into 24 sub-documents.


In [6]:
!pip install -U langchain-community
!pip install -U langchain langchain-core


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 22.6 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.43
    Uninstalling langsmith-0.3.43:
      Successfully uninstalled langsmith-0.3.43
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63


In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


<ipython-input-7-e5d49e4cc196>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 33.6 MB/s eta 0:00:00


In [9]:
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory

vectorstore = FAISS.from_documents(chunks, embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

# Initialize conversation memory
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")

# Domain detection function
def detect_domain(question):
    technical_terms = ['algorithm', 'programming', 'system', 'code', 'software', 'injection', 'pattern']
    academic_terms = ['theory', 'philosophy', 'research', 'study', 'principle']
    practical_terms = ['use', 'apply', 'implement', 'practice', 'how to']

    question_lower = question.lower()
    if any(term in question_lower for term in technical_terms):
        return "technical"
    elif any(term in question_lower for term in academic_terms):
        return "academic"
    elif any(term in question_lower for term in practical_terms):
        return "practical"
    return "general"

<ipython-input-9-8741112f2c43>:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")


In [10]:
from langchain.prompts import PromptTemplate

prompt_template = """
You are an expert Socratic tutor helping students learn. Your responses must:
1. ANALYZE the core concept in the question
2. IDENTIFY the most relevant domain (technical, academic, practical)
3. PROVIDE 1-2 concrete examples from real applications
4. ASK focused follow-up questions that advance understanding
5. NEVER give definitions or direct explanations

Conversation History:
{chat_history}

Relevant Context:
{context}

Student's Question:
{question}

Response Rules:
- First, identify the domain: {domain}
- For technical concepts: Reference specific technologies or implementations
- For academic concepts: Suggest experimental or theoretical angles
- For practical concepts: Use real-world scenarios
- Always end with a clear, specific question
- Maximum 3 sentences

Domain-Specific Examples:
- Technical: "In web development, how might this pattern solve cross-browser compatibility?"
- Academic: "What experimental results would validate this theory?"
- Practical: "How would this approach reduce costs in a manufacturing setting?"

Socratic Guidance:
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "context", "question", "domain"],
    template=prompt_template,
)

In [11]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_id = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)
llm = HuggingFacePipeline(pipeline=pipe)


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-11-23535542fadf>:9: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [12]:
from langchain.chains import LLMChain

# Step 1: create the manual chain
qa_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

<ipython-input-12-243bc7d47d5f>:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  qa_chain = LLMChain(


In [ ]:
# # # Step 2: create the ask_bot function manually
# # def ask_bot(question):
# #     docs = retriever.invoke(question)
# #     context = "\n\n".join([doc.page_content for doc in docs])
# #     result = qa_chain.run({"context": context, "question": question})
# #     filtered_result = enforce_hint_only(result, question)
# #     print("💡 Hint:")
# #     print(filtered_result)

# # def enforce_hint_only(response, question):
# #     forbidden_keywords = [
# #         "is a", "is the", "refers to", "means", "called", "defined as", "can be described as",
# #         "in simple terms", "in other words", "it is", "it’s", "this is", "this means"
# #     ]
# #     if any(keyword in response.lower() for keyword in forbidden_keywords):
# #         # Use the question to generate a more relevant hint
# #         return f"Let's think about '{question}' together. Can you recall a situation where you encountered this concept, or how it might be used in practice?"
# #     return response

# # Enhanced response filter
# def enforce_hint_only(response):
#     forbidden_phrases = [
#         "is a", "is the", "means", "defined as",
#         "refers to", "called", "is when", "is known as",
#         "the answer is", "basically", "essentially",
#         "in other words", "that is", "which means"
#     ]

#     # Check for definition patterns
#     if any(phrase in response.lower() for phrase in forbidden_phrases):
#         return "Interesting perspective! How might you apply this concept in a practical scenario?"

#     # Check for non-question responses
#     if not response.strip().endswith('?'):
#         return "What different approaches could we take to solve this?"

#     # Check for too much explanation
#     if len(response.split('.')) > 2:
#         return "Let me rephrase that as a question: What factors would influence this situation?"

#     return response

# # Modified ask_bot with memory support
# def ask_bot(question):
#     # Get relevant context
#     docs = retriever.invoke(question)
#     context = "\n\n".join([doc.page_content for doc in docs])

#     # Generate response with memory
#     raw_response = qa_chain.run(
#         context=context,
#         question=question,
#     )

#     # Apply strict filtering
#     filtered_response = enforce_hint_only(raw_response)

#     print("\n💡 Socratic Guidance:")
#     print(filtered_response)
#     print("\n---\n")  # Visual separator

# # Example conversation
# print("Tutor: I'll help you learn through questions. Ask me anything about design patterns!")
# while True:
#     user_input = input("You: ")
#     if user_input.lower() in ['exit', 'quit', 'bye']:
#         print("Tutor: Happy learning! Come back with more questions anytime.")
#         break
#     ask_bot(user_input)

Tutor: I'll help you learn through questions. Ask me anything about design patterns!


KeyboardInterrupt: Interrupted by user

In [ ]:
import random

# Enhanced response filter with domain awareness
def enforce_hint_only(response, question):
    # First detect domain
    domain = detect_domain(question)

    # Forbidden patterns
    forbidden_phrases = [
        "is a", "is the", "means", "defined as",
        "refers to", "called", "is when", "is known as",
        "the answer is", "basically", "essentially"
    ]

    # Domain-specific fallbacks
    domain_fallbacks = {
        "technical": [
            "How would this approach handle edge cases in a production environment?",
            "What performance tradeoffs might this implementation involve?",
            "Which system components would be most affected by this pattern?"
        ],
        "academic": [
            "What experimental design could test this hypothesis?",
            "How might recent research findings influence this theory?",
            "What competing explanations exist for this phenomenon?"
        ],
        "practical": [
            "How would you measure the success of this approach in a real project?",
            "What training would team members need to implement this effectively?",
            "Which stakeholders would be most impacted by this change?"
        ],
        "general": [
            "What's the most surprising application of this concept you've encountered?",
            "How might this idea evolve in the next 5 years?",
            "What personal experience relates to this concept?"
        ]
    }

    # Check for definition patterns
    if any(phrase in response.lower() for phrase in forbidden_phrases):
        return random.choice(domain_fallbacks[domain])

    # Check for non-question responses
    if not response.strip().endswith('?'):
        return random.choice(domain_fallbacks[domain])

    return response

# Final ask_bot implementation
def ask_bot(question):
    # Get relevant context
    docs = retriever.invoke(question)
    context = "\n\n".join([doc.page_content for doc in docs])

    # Detect domain for prompt
    domain = detect_domain(question)

    # Generate response
    raw_response = qa_chain.run(
        context=context,
        question=question,
        domain=domain
    )

    # Apply strict filtering
    filtered_response = enforce_hint_only(raw_response, question)

    print("\n🎯 Focused Guidance:")
    print(filtered_response)
    print("\n" + "="*50 + "\n")

# Example conversation
print("Tutor: I'll help you explore concepts through thoughtful questions. What would you like to discuss?")
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("Tutor: This was a productive discussion! Come back anytime to continue learning.")
        break
    ask_bot(user_input)

In [ ]:
ask_bot("what is dependency injection ?")

💡 Hint:
Let's think about 'what is dependency injection ?' together. Can you recall a situation where you encountered this concept, or how it might be used in practice?
